# 研修医の配属問題，安定マッチングアルゴリズム

[参考URL]

https://okumuralab.org/~okumura/python/stablematch.html

[参考文献]  

宮崎修一 『安定マッチングの数理とアルゴリズム』 （現代数学社,2018）
(2/26 図書館から借りる予定)

https://lib-kaishi-pu.opac.jp/opac/Free_word_search/hlist?q=%E5%AE%89%E5%AE%9A%E3%83%9E%E3%83%83%E3%83%81%E3%83%B3%E3%82%B0&tmtl=1&rgtn=000006976&idx=0


#  上記応用：「ICT総合活用実習」

**ゼミ生の研究室への配属問題の安定マッチング**

- 1:N割当 １つの研究室に定員N名の学生が割当可能
- 研究室の希望順序よりも，　学生の希望順序が優先される
- 学生の希望研究室リストについて，　どの学生から割当てもよい
- 学生は意図的に希望研究室リストを偽ったとしても得にならない


## サンプル小問題

-  研究室： 5
-  学生数： 20名
-  研究室の定員:  [4,4,4,4,4]  　# ５研究室　×　定員４名
-  研究室の選好 (Laboが希望するStudentの順序):  ランダム (1-20)　×　５研究室
-  学生の選好(Studentが希望するLaboの順序):  ランダム (1-5) 　×　２０学生

## プログラム　ロジックのおおまかな構成

- 各研究室の定員割当
- 乱数による希望順位作成 # 　本番は，CSVファイルからpd.DataFrame

- 問題設定: ``` problem() ```
- 安定マッチングによる学生の割当:  ```solve(); S2L();```
- 研究室からみた学生割当結果と学生からみた研究室割当結果を表示
- 割当結果の妥当性検証： ```verify()```
- 学生の選好がどれくらい満たされたか評価: ```eval_PS()```
- ラボの選好がどれくらい満たされたか評価: ```eval_PL()```



In [1]:
QUOTA = [0] + [4] * 5  #  各研究室の定員数　[0 4 4 4 4]

import copy
import numpy as np
np.random.seed(5731)  #  乱数の再現性のため
import pandas as pd
pd.options.display.float_format = '{:.0f}'.format

def shuffled(n):    # 乱数による順序生成
  a = np.arange(1,n+1)
  np.random.shuffle(a)
  return a.tolist()  # Python List

def problem():   # 　乱数を用いた問題設定
  lab_capa = QUOTA
  ns , nl = sum(lab_capa) , len(lab_capa)-1
  prf_s = [shuffled(nl) for _ in range(ns)]
  prf_l = [shuffled(ns) for _ in range(nl)]

  return [[]]+prf_s , [[]]+prf_l

#  リアルデータの場合は，CSVファイルからDataFrame生成

pS,pL = problem()

OpS = copy.deepcopy(pS) # 　学生の選好リストのオリジナルをバックアップ
OpL = copy.deepcopy(pL) # 　ラボの選好リストのオリジナルをバックアップ

df_s = pd.DataFrame(pS)
df_l = pd.DataFrame(pL)

display('学生の選好(Studentが希望するLaboの順序):ランダム(1-5)×２０学生')
display(df_s[1:])
display('研究室の選好 (Laboが希望するStudentの順序):ランダム(1-20)×５研究室')
display(df_l[1:])

display('リアルデータの場合は，CSVファイルからDataFrame生成')

# df_s = pd.read_csv('学生の選好リスト.csv')
# df_l = pd.read_csv('研究室の選好リスト.csv')


'学生の選好(Studentが希望するLaboの順序):ランダム(1-5)×２０学生'

,0,1,2,3,4
1,4,5,3,1,2
2,2,3,1,4,5
3,2,4,1,3,5
4,3,4,1,5,2
5,3,1,4,5,2
6,3,1,4,2,5
7,2,5,3,4,1
8,1,3,2,4,5
9,4,5,1,3,2
10,5,2,3,4,1


'研究室の選好 (Laboが希望するStudentの順序):ランダム(1-20)×５研究室'

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
1,4,7,6,15,3,12,16,8,11,5,14,10,20,9,18,1,13,2,17,19
2,20,14,11,18,7,16,12,3,17,9,2,6,19,5,15,8,4,1,10,13
3,19,2,16,4,20,6,11,9,10,3,15,17,18,13,1,5,14,12,8,7
4,7,15,16,9,19,4,6,14,20,2,10,18,11,8,3,12,5,13,1,17
5,12,3,15,5,14,19,10,6,20,7,17,2,1,11,13,8,4,18,16,9


'リアルデータの場合は，CSVファイルからDataFrame生成'

In [2]:
#  安定マッチングによる学生の割当　（学生の希望を優先）
Monitoring = True  # 割当プロセスの表示フラグ

def solve(A=pS, B=pL, quota=QUOTA):
    L2S = [[0]*q for q in quota]  # Lab -> Student

    for s in range(1, len(A)):
        while A[s]:
            t = A[s].pop(0)
            if Monitoring:
                print(f'Try: Student#{s} -> Labo#{t}')  # Process Monitoring
            if s in B[t]:
                if 0 in L2S[t]:  #  Not FULL
                    L2S[t][ L2S[t].index(0) ] = s
                    s = 0
                else:           #  FULL, need to exchange
                    k = max(range(len(L2S[t])), key=lambda x: B[t].index(L2S[t][x]))
                    if Monitoring:
                      print(f'FULL#{t} -> Overwrite {s}-->{L2S[t][k]}@[{t}][{k}]?')
                    if B[t].index(s) < B[t].index( L2S[t][k] ):
                        if Monitoring:
                           print(f'    Yes, Overwrite {s}-->{L2S[t][k]}@[{t}][{k}]!')
                        L2S[t][k], s = s, L2S[t][k]
                        print(f'Need to Re-Assign Student#{s}')
                print(L2S[1:])
    return L2S,A  # Lab -> Student   研究室視点：　　割り当てられた学生リスト

def S2L(L2S,a):   # Student -> Lab   学生視点：　割り当てられた研究室リスト
    s2l = [0] * len(a)
    for t,s in enumerate(L2S):
        for k in s:
            s2l[k] = t
    print(s2l[1:])
    return L2S[1:] , s2l[1:]  # Lab -> Student

L2s , S2l = S2L(*solve())

dfL = pd.DataFrame(L2s)
display('研究室視点：　割り当てられた学生リスト')
display(dfL)

dfS = pd.DataFrame(S2l)
display('学生視点：割り当てられた研究室リスト')
display(dfS)



Try: Student#1 -> Labo#4
[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 0]]
Try: Student#2 -> Labo#2
[[0, 0, 0, 0], [2, 0, 0, 0], [0, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 0]]
Try: Student#3 -> Labo#2
[[0, 0, 0, 0], [2, 3, 0, 0], [0, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 0]]
Try: Student#4 -> Labo#3
[[0, 0, 0, 0], [2, 3, 0, 0], [4, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 0]]
Try: Student#5 -> Labo#3
[[0, 0, 0, 0], [2, 3, 0, 0], [4, 5, 0, 0], [1, 0, 0, 0], [0, 0, 0, 0]]
Try: Student#6 -> Labo#3
[[0, 0, 0, 0], [2, 3, 0, 0], [4, 5, 6, 0], [1, 0, 0, 0], [0, 0, 0, 0]]
Try: Student#7 -> Labo#2
[[0, 0, 0, 0], [2, 3, 7, 0], [4, 5, 6, 0], [1, 0, 0, 0], [0, 0, 0, 0]]
Try: Student#8 -> Labo#1
[[8, 0, 0, 0], [2, 3, 7, 0], [4, 5, 6, 0], [1, 0, 0, 0], [0, 0, 0, 0]]
Try: Student#9 -> Labo#4
[[8, 0, 0, 0], [2, 3, 7, 0], [4, 5, 6, 0], [1, 9, 0, 0], [0, 0, 0, 0]]
Try: Student#10 -> Labo#5
[[8, 0, 0, 0], [2, 3, 7, 0], [4, 5, 6, 0], [1, 9, 0, 0], [10, 0, 0, 0]]
Try: Student#11 -> Labo#5
[[8, 0, 0, 0

'研究室視点：\u3000割り当てられた学生リスト'

,0,1,2,3
0,8,15,14,5
1,16,18,7,12
2,4,2,6,20
3,1,9,3,13
4,10,11,17,19


'学生視点：割り当てられた研究室リスト'

,0
0,4
1,3
2,4
3,3
4,1
5,3
6,2
7,1
8,4
9,5


In [3]:
##  アサイン結果の妥当性の検証
##  print(L2s)  # 研究室視点：　割り当てられた学生リスト
##  print(S2l)   # 学生視点：　割り当てられた研究室リスト

# ラボの定員制約を満たしているか？
def q2lst(quota=QUOTA):
  result = []
  for i in range(1,len(quota)):
    n = quota[i]
    result += ([i] * n)
  return result

def verify( l2s=L2s, s2l=S2l, quota=QUOTA):
  alls = [ v for x in l2s for v in x ]
  print(alls)
  set_alls = set(alls)
  print(set_alls)
  assert len(alls) == len(set_alls) # 全員が，重複なく１つの研究室にアサイン済か？

  lab_all = sorted(s2l)
  print(lab_all)
  assert lab_all == q2lst() # 全てのラボに定員漏れ・溢れなく学生がアサイン済か？

verify()

[8, 15, 14, 5, 16, 18, 7, 12, 4, 2, 6, 20, 1, 9, 3, 13, 10, 11, 17, 19]
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20}
[1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5]


In [4]:
def eval_PS(s2l=S2l, ops=OpS): # Studentsの選好がどれくらい満たされたか評価する
  order = []
  for i in range(1,len(s2l)+1):
    result = s2l[i-1]
    pref = ops[i]
    rank = pref.index(result)+1
    order += [rank]
    print(f'Student#{i} prefs {pref} --> results {result}; orders:{rank}')
  return order # 配属結果に対する選好順位のリスト

orders = eval_PS()
print('学生からみた配属研究所の希望順位の全体平均：')
sum(orders)/len(orders)

Student#1 prefs [4, 5, 3, 1, 2] --> results 4; orders:1
Student#2 prefs [2, 3, 1, 4, 5] --> results 3; orders:2
Student#3 prefs [2, 4, 1, 3, 5] --> results 4; orders:2
Student#4 prefs [3, 4, 1, 5, 2] --> results 3; orders:1
Student#5 prefs [3, 1, 4, 5, 2] --> results 1; orders:2
Student#6 prefs [3, 1, 4, 2, 5] --> results 3; orders:1
Student#7 prefs [2, 5, 3, 4, 1] --> results 2; orders:1
Student#8 prefs [1, 3, 2, 4, 5] --> results 1; orders:1
Student#9 prefs [4, 5, 1, 3, 2] --> results 4; orders:1
Student#10 prefs [5, 2, 3, 4, 1] --> results 5; orders:1
Student#11 prefs [5, 3, 4, 1, 2] --> results 5; orders:1
Student#12 prefs [3, 2, 5, 1, 4] --> results 2; orders:2
Student#13 prefs [5, 1, 3, 2, 4] --> results 4; orders:5
Student#14 prefs [3, 1, 2, 4, 5] --> results 1; orders:2
Student#15 prefs [2, 1, 4, 3, 5] --> results 1; orders:2
Student#16 prefs [5, 2, 3, 1, 4] --> results 2; orders:2
Student#17 prefs [3, 2, 5, 1, 4] --> results 5; orders:3
Student#18 prefs [1, 3, 5, 2, 4] --> res

1.85

In [5]:
def eval_PL(l2s=L2s,opl=OpL[1:]):  # Labsの選好がどれくらい満たされたか評価する
  orders = []
  for i in range(len(l2s)):
    pref , result = opl[i] , l2s[i]
    rank = []
    for r in result:
      rank += [pref.index(r)+1]
    print(f'Lab#{i+1} prefs {pref} --> results {result}/orders{rank}; average={sum(rank)/len(rank)}')
    orders += [sorted(rank)]
  return orders  # 配属結果に対する選好順位のリスト

orders = eval_PL()
print('ラボからみた配属学生の希望順位の全体平均：')
sum([sum(s) for s in orders])/sum(QUOTA)

Lab#1 prefs [4, 7, 6, 15, 3, 12, 16, 8, 11, 5, 14, 10, 20, 9, 18, 1, 13, 2, 17, 19] --> results [8, 15, 14, 5]/orders[8, 4, 11, 10]; average=8.25
Lab#2 prefs [20, 14, 11, 18, 7, 16, 12, 3, 17, 9, 2, 6, 19, 5, 15, 8, 4, 1, 10, 13] --> results [16, 18, 7, 12]/orders[6, 4, 5, 7]; average=5.5
Lab#3 prefs [19, 2, 16, 4, 20, 6, 11, 9, 10, 3, 15, 17, 18, 13, 1, 5, 14, 12, 8, 7] --> results [4, 2, 6, 20]/orders[4, 2, 6, 5]; average=4.25
Lab#4 prefs [7, 15, 16, 9, 19, 4, 6, 14, 20, 2, 10, 18, 11, 8, 3, 12, 5, 13, 1, 17] --> results [1, 9, 3, 13]/orders[19, 4, 15, 18]; average=14.0
Lab#5 prefs [12, 3, 15, 5, 14, 19, 10, 6, 20, 7, 17, 2, 1, 11, 13, 8, 4, 18, 16, 9] --> results [10, 11, 17, 19]/orders[7, 14, 11, 6]; average=9.5
ラボからみた配属学生の希望順位の全体平均：


8.3

# 現実的なスケールの問題設定

- 研究室: １４
- 学生数: ６７名
- 研究室の定員： [5] * 12 + [4,3]   # [5 5 5 5 5 5 5 5 5 5 5 5 4 3]
- 研究室の選好 (Laboが希望するStudentの順序)： ランダム(1-67) ×　14研究室
- 学生の選好(Studentが希望するLaboの順序)： ランダム(1-14) ×　67学生

QUOTA = [0] + [5] * 12 + [4,3]


**ゼミ生の研究室への配属問題の安定マッチング**

- 1:N割当 １つの研究室に定員N名の学生が割当可能
- 研究室の希望順序よりも，　学生の希望順序が優先される
- 学生の希望研究室リストについて，　どの学生から割当てもよい
- 学生は意図的に希望研究室リストを偽ったとしても得にならない


## プログラム　ロジックのおおまかな構成

- 各研究室の定員割当
- 乱数による希望順位作成 # 　本番は，CSVファイルからpd.DataFrame

- 問題設定: ``` problem() ```
- 安定マッチングによる学生の割当:  ```solve(); S2L();```
- 研究室からみた学生割当結果と学生からみた研究室割当結果を表示
- 割当結果の妥当性検証： ```verify()```
- 学生の選好がどれくらい満たされたか評価: ```eval_PS()```
- ラボの選好がどれくらい満たされたか評価: ```eval_PL()```




In [6]:
QUOTA = [0] + [5] * 12 + [4,3]

import copy
import numpy as np
np.random.seed(5731)  #  乱数の再現性のため
import pandas as pd
pd.options.display.float_format = '{:.0f}'.format

def shuffled(n):    # 乱数による順序生成
  a = np.arange(1,n+1)
  np.random.shuffle(a)
  return a.tolist()  # Python List

def problem():   # 　乱数を用いた問題設定
  lab_capa = QUOTA
  ns , nl = sum(lab_capa) , len(lab_capa)-1
  prf_s = [shuffled(nl) for _ in range(ns)]
  prf_l = [shuffled(ns) for _ in range(nl)]

  return [[]]+prf_s , [[]]+prf_l

#  リアルデータの場合は，CSVファイルからDataFrame生成

pS,pL = problem()

OpS = copy.deepcopy(pS) # 　学生の選好リストのオリジナルをバックアップ
OpL = copy.deepcopy(pL) # 　ラボの選好リストのオリジナルをバックアップ

df_s = pd.DataFrame(pS)
df_l = pd.DataFrame(pL)

display('学生の選好(Studentが希望するLaboの順序): ランダム(1-5)×２０学生')
display(df_s[1:])
display('研究室の選好 (Laboが希望するStudentの順序): ランダム(1-20)×５研究室')
display(df_l[1:])

display('リアルデータの場合は，CSVファイルからDataFrame生成')

# df_s = pd.read_csv('学生の選好リスト.csv')
# df_l = pd.read_csv('研究室の選好リスト.csv')


'学生の選好(Studentが希望するLaboの順序): ランダム(1-5)×２０学生'

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
1,12,4,14,6,2,11,3,5,9,13,8,7,1,10
2,4,6,11,7,5,8,1,3,13,12,9,14,10,2
3,7,10,4,12,1,11,14,6,13,9,5,8,3,2
4,1,2,13,9,3,11,12,6,8,5,7,10,4,14
5,1,7,3,10,2,13,8,6,5,14,4,12,11,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,14,8,5,2,11,10,12,6,9,1,4,7,3,13
64,5,3,11,2,1,13,4,6,10,14,12,7,8,9
65,12,2,5,4,8,6,3,14,13,10,7,1,9,11
66,1,11,14,8,9,4,3,5,10,12,6,2,13,7


'研究室の選好 (Laboが希望するStudentの順序): ランダム(1-20)×５研究室'

,0,1,2,3,4,5,6,7,8,9,...,57,58,59,60,61,62,63,64,65,66
1,16,19,31,21,22,62,30,66,41,28,...,35,54,45,59,26,43,64,14,33,9
2,45,52,62,23,30,24,8,29,51,60,...,49,33,5,4,14,3,12,21,63,15
3,39,43,33,10,54,18,3,5,50,4,...,21,57,9,47,8,36,23,16,42,58
4,9,48,35,10,25,49,28,26,4,20,...,45,57,27,31,6,32,14,36,42,24
5,64,9,3,42,28,54,22,1,4,34,...,38,19,46,53,66,13,24,31,50,47
6,57,53,9,47,5,55,20,4,26,58,...,49,48,1,51,14,24,45,37,22,25
7,29,10,46,58,20,48,52,35,43,15,...,40,27,57,21,39,47,59,4,38,26
8,66,35,27,30,39,52,29,37,51,65,...,64,18,62,5,43,25,8,19,67,9
9,27,7,17,26,12,19,30,10,47,51,...,39,28,2,63,55,8,20,50,46,24
10,32,18,54,42,56,41,33,4,59,8,...,11,21,35,63,22,34,45,58,60,51


'リアルデータの場合は，CSVファイルからDataFrame生成'

In [7]:
#  安定マッチングによる学生の割当　（学生の希望を優先）
Monitoring = True  # 割当プロセスの表示フラグ

def solve(A=pS, B=pL, quota=QUOTA):
    L2S = [[0]*q for q in quota]  # Lab -> Student

    for s in range(1, len(A)):
        while A[s]:
            t = A[s].pop(0)
            if Monitoring:
                print(f'Try: Student#{s} -> Labo#{t}')  # Process Monitoring
            if s in B[t]:
                if 0 in L2S[t]:  #  Not FULL
                    L2S[t][ L2S[t].index(0) ] = s
                    s = 0
                else:           #  FULL, need to exchange
                    k = max(range(len(L2S[t])), key=lambda x: B[t].index(L2S[t][x]))
                    if Monitoring:
                      print(f'FULL#{t} -> Overwrite {s}-->{L2S[t][k]}@[{t}][{k}]?')
                    if B[t].index(s) < B[t].index( L2S[t][k] ):
                        if Monitoring:
                           print(f'    Yes, Overwrite {s}-->{L2S[t][k]}@[{t}][{k}]!')
                        L2S[t][k], s = s, L2S[t][k]
                        print(f'Need to Re-Assign Student#{s}')
                print(L2S[1:])
    return L2S,A  # Lab -> Student   研究室視点：　　割り当てられた学生リスト

def S2L(L2S,a):   # Student -> Lab   学生視点：　割り当てられた研究室リスト
    s2l = [0] * len(a)
    for t,s in enumerate(L2S):
        for k in s:
            s2l[k] = t
    print(s2l[1:])
    return L2S[1:] , s2l[1:]  # Lab -> Student

L2s , S2l = S2L(*solve())

dfL = pd.DataFrame(L2s)
display('研究室視点：　割り当てられた学生リスト')
display(dfL)

dfS = pd.DataFrame(S2l)
display('学生視点：割り当てられた研究室リスト')
display(dfS)


Try: Student#1 -> Labo#12
[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0]]
Try: Student#2 -> Labo#4
[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [2, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0]]
Try: Student#3 -> Labo#7
[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [2, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [3, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0]]
Try: Student#4 -> Labo#1
[[4, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [2, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [3, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [1, 0, 0, 0, 0], [0, 

'研究室視点：\u3000割り当てられた学生リスト'

,0,1,2,3,4
0,4,29,62,67,66
1,51,27,32,9,35
2,33,14,5,52,59
3,38,7,26,28,37
4,64,6,34,40,43
5,15,21,12,2,42
6,36,13,17,44,45
7,11,65,58,60,49
8,16,30,55,31,39
9,61,23,54,41,48


'学生視点：割り当てられた研究室リスト'

,0
0,12
1,6
2,12
3,1
4,3
...,...
62,14
63,5
64,8
65,1


In [8]:
##  アサイン結果の妥当性の検証
##  print(L2s)  # 研究室視点：　割り当てられた学生リスト
##  print(S2l)   # 学生視点：　割り当てられた研究室リスト

# ラボの定員制約を満たしているか？
def q2lst(quota=QUOTA):
  result = []
  for i in range(1,len(quota)):
    n = quota[i]
    result += ([i] * n)
  return result

def verify( l2s=L2s, s2l=S2l, quota=QUOTA):
  alls = [ v for x in l2s for v in x ]
  print(alls)
  set_alls = set(alls)
  print(set_alls)
  assert len(alls) == len(set_alls) # 全員が，重複なく１つの研究室にアサイン済か？

  lab_all = sorted(s2l)
  print(lab_all)
  assert lab_all == q2lst() # 全てのラボに定員漏れ・溢れなく学生がアサイン済か？

verify()


[4, 29, 62, 67, 66, 51, 27, 32, 9, 35, 33, 14, 5, 52, 59, 38, 7, 26, 28, 37, 64, 6, 34, 40, 43, 15, 21, 12, 2, 42, 36, 13, 17, 44, 45, 11, 65, 58, 60, 49, 16, 30, 55, 31, 39, 61, 23, 54, 41, 48, 10, 22, 57, 19, 24, 1, 20, 3, 47, 53, 25, 50, 56, 46, 8, 18, 63]
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67}
[1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 13, 13, 13, 13, 14, 14, 14]


In [9]:
def eval_PS(s2l=S2l, ops=OpS): # Studentsの選好がどれくらい満たされたか評価する
  order = []
  for i in range(1,len(s2l)+1):
    result = s2l[i-1]
    pref = ops[i]
    rank = pref.index(result)+1
    order += [rank]
    print(f'Student#{i} prefs {pref} --> results {result}; orders:{rank}')
  return order # 配属結果に対する選好順位のリスト

orders = eval_PS()
sum(orders)/len(orders)

Student#1 prefs [12, 4, 14, 6, 2, 11, 3, 5, 9, 13, 8, 7, 1, 10] --> results 12; orders:1
Student#2 prefs [4, 6, 11, 7, 5, 8, 1, 3, 13, 12, 9, 14, 10, 2] --> results 6; orders:2
Student#3 prefs [7, 10, 4, 12, 1, 11, 14, 6, 13, 9, 5, 8, 3, 2] --> results 12; orders:4
Student#4 prefs [1, 2, 13, 9, 3, 11, 12, 6, 8, 5, 7, 10, 4, 14] --> results 1; orders:1
Student#5 prefs [1, 7, 3, 10, 2, 13, 8, 6, 5, 14, 4, 12, 11, 9] --> results 3; orders:3
Student#6 prefs [10, 7, 5, 2, 6, 1, 14, 13, 11, 4, 9, 8, 12, 3] --> results 5; orders:3
Student#7 prefs [4, 7, 6, 14, 3, 12, 1, 8, 11, 5, 13, 10, 9, 2] --> results 4; orders:1
Student#8 prefs [14, 12, 9, 6, 2, 5, 7, 8, 4, 3, 11, 1, 10, 13] --> results 14; orders:1
Student#9 prefs [1, 12, 2, 13, 5, 9, 7, 3, 10, 8, 6, 14, 4, 11] --> results 2; orders:3
Student#10 prefs [11, 4, 2, 9, 6, 13, 3, 14, 10, 1, 12, 5, 8, 7] --> results 11; orders:1
Student#11 prefs [2, 10, 8, 9, 3, 14, 5, 7, 12, 11, 13, 6, 1, 4] --> results 8; orders:3
Student#12 prefs [3, 6, 9,

1.8656716417910448

In [10]:
def eval_PL(l2s=L2s,opl=OpL[1:]):  # Labsの選好がどれくらい満たされたか評価する
  orders = []
  for i in range(len(l2s)):
    pref , result = opl[i] , l2s[i]
    rank = []
    for r in result:
      rank += [pref.index(r)+1]
    print(f'Lab#{i+1} prefs {pref} --> results {result}/orders{rank}; average={sum(rank)/len(rank)}')
    orders += [sorted(rank)]
  return orders  # 配属結果に対する選好順位のリスト

orders = eval_PL()
print('ラボからみた配属学生の希望順位の全体平均：')
sum([sum(s) for s in orders])/sum(QUOTA)

Lab#1 prefs [16, 19, 31, 21, 22, 62, 30, 66, 41, 28, 60, 6, 56, 29, 27, 10, 37, 15, 2, 4, 63, 11, 58, 53, 18, 34, 67, 49, 44, 25, 39, 48, 38, 50, 3, 12, 51, 42, 23, 1, 46, 40, 65, 55, 20, 5, 7, 24, 36, 32, 57, 17, 61, 52, 8, 47, 13, 35, 54, 45, 59, 26, 43, 64, 14, 33, 9] --> results [4, 29, 62, 67, 66]/orders[20, 14, 6, 27, 8]; average=15.0
Lab#2 prefs [45, 52, 62, 23, 30, 24, 8, 29, 51, 60, 39, 19, 10, 35, 37, 9, 66, 59, 32, 6, 40, 26, 27, 64, 20, 38, 2, 31, 67, 58, 34, 11, 44, 1, 17, 13, 16, 22, 61, 48, 50, 47, 36, 7, 65, 28, 55, 18, 25, 56, 42, 54, 43, 41, 53, 57, 46, 49, 33, 5, 4, 14, 3, 12, 21, 63, 15] --> results [51, 27, 32, 9, 35]/orders[9, 23, 19, 16, 14]; average=16.2
Lab#3 prefs [39, 43, 33, 10, 54, 18, 3, 5, 50, 4, 30, 40, 44, 37, 13, 48, 27, 26, 46, 19, 29, 28, 17, 14, 38, 2, 60, 22, 56, 24, 41, 63, 65, 15, 31, 53, 35, 62, 6, 55, 34, 61, 20, 49, 25, 59, 11, 67, 52, 51, 12, 64, 1, 45, 7, 66, 32, 21, 57, 9, 47, 8, 36, 23, 16, 42, 58] --> results [33, 14, 5, 52, 59]/orders[3,

22.37313432835821